In [ ]:
import os
import json
from typing import List, Dict, Any

import pandas as pd

# ==============================
# 0. 경로 및 기본 설정
# ==============================

COHORT_PATH = "./cohort/cohort_ver50_only_subject_id.csv"

HOSP_DIR = "../../data/MIMIC4-hosp-icu"
ICU_DIR = "../../data/MIMIC4-hosp-icu"
ED_DIR = "../../data/mimic-iv-ed/ed"
ECG_DIR = "../../data/mimic-iv-ecg"

OUTPUT_DIR = "./cohort"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 전체 Event Log를 하나로 합친 파일
EVENT_LOG_FULL_PATH = os.path.join(OUTPUT_DIR, "event_log_stemi_all.csv")

# 중간 저장 chunk 파일 이름 규칙
CHUNK_VERSION_START = 140
BATCH_SIZE = 100  # 한 번에 처리할 환자 수

# Troponin 양성 기준 (예: 0.04 이상 양성)
TROP_POS_THRESHOLD = 0.04

# PCI ICD 코드 prefix (예시 – 필요시 조정 가능)
PCI_ICD9_PREFIXES = ["00.66", "36.0"]
PCI_ICD10_PREFIXES = ["027", "929"]

# 항혈소판제 이름 리스트 (대소문자 무시)
ANTI_PLT_DRUGS = ["aspirin", "clopidogrel", "ticagrelor", "prasugrel"]


# ==============================
# 1. 공통 유틸
# ==============================

def _to_datetime(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """
    datetime 파싱.
    연도 이상치(미래 연도)는 그대로 허용하고, 파싱 실패만 NaT 처리.
    """
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df


def _attrs_to_json(attrs: Dict[str, Any]) -> str:
    """attributes dict를 JSON 문자열로 변환."""
    clean = {}
    for k, v in attrs.items():
        if isinstance(v, float) and pd.isna(v):
            clean[k] = None
        else:
            clean[k] = v
    return json.dumps(clean, ensure_ascii=False)


# ==============================
# 2. Cohort 및 원본 테이블 로딩
# ==============================

def load_labevents_troponin() -> pd.DataFrame:
    """
    Troponin 전용 labevents_troponin.csv 로딩.
    - HOSP_DIR 아래 또는 현재 작업 디렉토리에서 탐색.
    """
    candidates = [
        os.path.join(HOSP_DIR, "labevents_troponin.csv"),
        "./labevents_troponin.csv",
    ]
    for p in candidates:
        if os.path.exists(p):
            print(f"[LOAD] labevents_troponin.csv 로딩: {p}")
            df = pd.read_csv(p)
            return df

    raise FileNotFoundError(
        "labevents_troponin.csv 파일을 찾을 수 없습니다.\n"
        f"다음 경로 중 하나에 존재해야 합니다.\n{candidates}"
    )


def load_cohort_all_hadm(path: str) -> pd.DataFrame:
    """
    cohort 파일에서 (subject_id, hadm_id) 모든 조합을 사용.
    - 같은 (subject_id, hadm_id)는 중복만 제거.
    필수 컬럼: subject_id, hadm_id
    """
    cohort_raw = pd.read_csv(path)

    if "subject_id" not in cohort_raw.columns:
        raise ValueError(f"cohort 파일에 subject_id 컬럼이 없습니다. 현재 컬럼: {list(cohort_raw.columns)}")
    if "hadm_id" not in cohort_raw.columns:
        raise ValueError(
            "cohort 파일에 hadm_id 컬럼이 없습니다.\n"
            "STEMI 분석은 hadm_id(입원 단위) 기준으로 해야 합니다."
        )

    print(f"[COHORT] 입력 cohort row 수: {len(cohort_raw)}")
    print(f"[COHORT] 입력 cohort 고유 subject 수: {cohort_raw['subject_id'].nunique()}")
    print(f"[COHORT] 입력 cohort 고유 hadm 수: {cohort_raw['hadm_id'].nunique()}")

    cohort_final = cohort_raw[["subject_id", "hadm_id"]].drop_duplicates().reset_index(drop=True)

    print(f"[COHORT] (subject_id, hadm_id) 중복 제거 후 row 수: {len(cohort_final)}")
    print(f"[COHORT] 최종 고유 subject 수: {cohort_final['subject_id'].nunique()}")
    print(f"[COHORT] 최종 고유 hadm 수: {cohort_final['hadm_id'].nunique()}")

    return cohort_final


def load_source_tables() -> Dict[str, pd.DataFrame]:
    """
    MIMIC-IV 원본 테이블들을 로딩.
    - Troponin은 labevents_troponin.csv만 사용.
    """
    print("[LOAD] admissions, patients, icustays...")
    admissions = pd.read_csv(os.path.join(HOSP_DIR, "admissions.csv"))
    patients = pd.read_csv(os.path.join(HOSP_DIR, "patients.csv"))
    icustays = pd.read_csv(os.path.join(ICU_DIR, "icustays.csv"))

    print("[LOAD] edstays (있으면 로딩)...")
    edstays_path = os.path.join(ED_DIR, "edstays.csv")
    edstays = pd.read_csv(edstays_path) if os.path.exists(edstays_path) else None

    print("[LOAD] labevents_troponin ...")
    labevents_trop = load_labevents_troponin()

    print("[LOAD] procedures_icd, prescriptions, ECG machine_measurements...")
    procedures_icd = pd.read_csv(os.path.join(HOSP_DIR, "procedures_icd.csv"))
    prescriptions = pd.read_csv(os.path.join(HOSP_DIR, "prescriptions.csv"))
    ecg = pd.read_csv(os.path.join(ECG_DIR, "machine_measurements.csv"))

    return {
        "admissions": admissions,
        "patients": patients,
        "icustays": icustays,
        "edstays": edstays,
        "labevents_trop": labevents_trop,
        "procedures_icd": procedures_icd,
        "prescriptions": prescriptions,
        "ecg": ecg,
    }


# ==============================
# 3. Event 생성 함수들
# ==============================

def build_real_ed_events(cohort: pd.DataFrame,
                         edstays: pd.DataFrame) -> pd.DataFrame:
    """
    실제 ED_ARRIVAL, ED_DEPARTURE 이벤트 생성 (edstays 기반).
    surrogate는 여기에서 만들지 않고, 나중에 전체 이벤트를 본 뒤
    '최초 timestamp' 기반으로 ED_ARRIVAL_SURR를 생성한다.
    """
    if edstays is None:
        return pd.DataFrame(columns=["subject_id", "hadm_id", "event_name", "timestamp", "attributes"])

    ed = edstays.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    ed = _to_datetime(ed, ["intime", "outtime"])

    events = []
    for _, row in ed.iterrows():
        sid = row["subject_id"]
        hid = row["hadm_id"]

        if pd.notnull(row.get("intime")):
            events.append({
                "subject_id": sid,
                "hadm_id": hid,
                "event_name": "ED_ARRIVAL",
                "timestamp": row["intime"],
                "attributes": _attrs_to_json({}),
            })
        if pd.notnull(row.get("outtime")):
            events.append({
                "subject_id": sid,
                "hadm_id": hid,
                "event_name": "ED_DEPARTURE",
                "timestamp": row["outtime"],
                "attributes": _attrs_to_json({}),
            })

    if not events:
        return pd.DataFrame(columns=["subject_id", "hadm_id", "event_name", "timestamp", "attributes"])

    df = pd.DataFrame(events)
    df = _to_datetime(df, ["timestamp"])
    return df


def build_admission_events(cohort: pd.DataFrame,
                           admissions: pd.DataFrame,
                           patients: pd.DataFrame) -> pd.DataFrame:
    """
    DISCHARGE, DEATH 이벤트 생성.
    DISCHARGE: admissions.dischtime
    DEATH: patients.dod
    """
    adm = admissions.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    adm = _to_datetime(adm, ["admittime", "dischtime"])
    pat = _to_datetime(patients.copy(), ["dod"])

    adm = adm.merge(pat[["subject_id", "dod"]], on="subject_id", how="left")

    events = []
    for _, row in adm.iterrows():
        if pd.notnull(row.get("dischtime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "DISCHARGE",
                "timestamp": row["dischtime"],
                "attributes": _attrs_to_json({}),
            })
        if pd.notnull(row.get("dod")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "DEATH",
                "timestamp": row["dod"],
                "attributes": _attrs_to_json({}),
            })

    return pd.DataFrame(events)


def build_icu_events(cohort: pd.DataFrame, icustays: pd.DataFrame) -> pd.DataFrame:
    """
    ICU_INTIME, ICU_OUTTIME 이벤트 생성.
    icustays: subject_id, hadm_id, intime, outtime, first_careunit, last_careunit, stay_id 가정.
    """
    icu = icustays.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    icu = _to_datetime(icu, ["intime", "outtime"])

    events = []
    for _, row in icu.iterrows():
        attrs = {
            "first_careunit": row.get("first_careunit", None),
            "last_careunit": row.get("last_careunit", None),
            "stay_id": row.get("stay_id", None),
        }
        if pd.notnull(row.get("intime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ICU_INTIME",
                "timestamp": row["intime"],
                "attributes": _attrs_to_json(attrs),
            })
        if pd.notnull(row.get("outtime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ICU_OUTTIME",
                "timestamp": row["outtime"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


def build_troponin_events(cohort: pd.DataFrame,
                          labevents_trop: pd.DataFrame,
                          positive_threshold: float) -> pd.DataFrame:
    """
    TROP_TAKEN, TROP_POSITIVE 이벤트 생성.
    labevents_trop: Troponin만 필터된 labevents_troponin.csv
      (subject_id, hadm_id, itemid, charttime, valuenum 등 포함)
    """
    lab = labevents_trop.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    lab = _to_datetime(lab, ["charttime"])

    events = []

    # TROP_TAKEN
    for _, row in lab.iterrows():
        if pd.notnull(row.get("charttime")):
            attrs = {
                "itemid": row.get("itemid", None),
                "valuenum": row.get("valuenum", None),
                "value": row.get("value", None),
                "flag": row.get("flag", None),
            }
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "TROP_TAKEN",
                "timestamp": row["charttime"],
                "attributes": _attrs_to_json(attrs),
            })

    # TROP_POSITIVE: hadm_id 기준 첫 양성 시점
    if "valuenum" in lab.columns:
        lab_pos = lab[lab["valuenum"] >= positive_threshold].copy()
        lab_pos = lab_pos.dropna(subset=["charttime"])
        lab_pos_sorted = lab_pos.sort_values(["subject_id", "hadm_id", "charttime"])
        first_pos = lab_pos_sorted.groupby(["subject_id", "hadm_id"], as_index=False).head(1)

        for _, row in first_pos.iterrows():
            attrs = {
                "itemid": row.get("itemid", None),
                "valuenum": row.get("valuenum", None),
                "value": row.get("value", None),
                "flag": row.get("flag", None),
            }
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "TROP_POSITIVE",
                "timestamp": row["charttime"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


def build_ecg_events(cohort: pd.DataFrame, ecg: pd.DataFrame) -> pd.DataFrame:
    """
    ECG_TAKEN, ECG_STEMI_FLAG 이벤트 생성

    - machine_measurements.csv는 보통 hadm_id가 없고 subject_id만 있음.
    - subject_id로 cohort와 join해서 hadm_id를 붙임.
    - 시간 컬럼은 ecg_time 또는 charttime 중 존재하는 것을 사용.
    """
    ecg_c = ecg.copy()
    if "charttime" not in ecg_c.columns:
        if "ecg_time" in ecg_c.columns:
            ecg_c = ecg_c.rename(columns={"ecg_time": "charttime"})
        else:
            raise ValueError(
                "ECG(machine_measurements) 데이터에 charttime/ecg_time 둘 다 없습니다. "
                "시간 컬럼 이름을 확인해 주세요."
            )

    subj_hadm_map = cohort[["subject_id", "hadm_id"]].drop_duplicates()
    ecg_c = ecg_c.merge(subj_hadm_map, on="subject_id", how="inner")
    ecg_c = _to_datetime(ecg_c, ["charttime"])

    def has_stemi_flag(row) -> bool:
        mm = str(row.get("machine_measurements", "")).upper()
        reports = []
        for i in range(30):
            col = f"report_{i}"
            if col in row.index:
                reports.append(str(row.get(col, "")))
        rep_text = " ".join(reports).upper()
        text = mm + " " + rep_text

        if "STEMI" in text:
            return True
        if "ST ELEVATION" in text:
            return True
        return False

    ecg_c["is_stemi"] = ecg_c.apply(has_stemi_flag, axis=1)

    events = []
    for _, row in ecg_c.iterrows():
        ts = row.get("charttime")
        if pd.isna(ts):
            continue

        events.append({
            "subject_id": row["subject_id"],
            "hadm_id": row["hadm_id"],
            "event_name": "ECG_TAKEN",
            "timestamp": ts,
            "attributes": _attrs_to_json({}),
        })

        if row["is_stemi"]:
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ECG_STEMI_FLAG",
                "timestamp": ts,
                "attributes": _attrs_to_json({}),
            })

    return pd.DataFrame(events)


def build_pci_events(cohort: pd.DataFrame,
                     procedures_icd: pd.DataFrame,
                     pci_icd9_prefixes: List[str],
                     pci_icd10_prefixes: List[str]) -> pd.DataFrame:
    """
    PCI_START 이벤트 생성 (날짜 단위).
    procedures_icd: subject_id, hadm_id, icd_code, icd_version, chartdate 가정.
    """
    proc = procedures_icd.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    proc = _to_datetime(proc, ["chartdate"])

    def is_pci(code: Any, version: Any) -> bool:
        if pd.isna(code):
            return False
        code_str = str(code)
        if version == 9:
            return any(code_str.startswith(p) for p in pci_icd9_prefixes)
        elif version == 10:
            return any(code_str.startswith(p) for p in pci_icd10_prefixes)
        else:
            return False

    proc["is_pci"] = proc.apply(
        lambda r: is_pci(r.get("icd_code", None), r.get("icd_version", None)), axis=1
    )
    pci_rows = proc[proc["is_pci"]].copy()

    events = []
    for _, row in pci_rows.iterrows():
        ts = row.get("chartdate")
        if pd.isna(ts):
            continue
        attrs = {
            "icd_code": row.get("icd_code", None),
            "icd_version": row.get("icd_version", None),
        }
        events.append({
            "subject_id": row["subject_id"],
            "hadm_id": row["hadm_id"],
            "event_name": "PCI_START",
            "timestamp": ts,
            "attributes": _attrs_to_json(attrs),
        })

    return pd.DataFrame(events)


def build_antiplatelet_events(cohort: pd.DataFrame,
                              prescriptions: pd.DataFrame,
                              drug_name_list: List[str]) -> pd.DataFrame:
    """
    ANTI_PLT_ORDER, ANTI_PLT_ADMIN 이벤트 생성.
    prescriptions: subject_id, hadm_id, drug, starttime, stoptime, route, dose_val_rx, dose_unit_rx 가정.
    """
    rx = prescriptions.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    rx = _to_datetime(rx, ["starttime", "stoptime"])

    drug_lower_list = [d.lower() for d in drug_name_list]
    rx["drug_lower"] = rx["drug"].astype(str).str.lower()
    rx = rx[rx["drug_lower"].isin(drug_lower_list)]

    events = []
    for _, row in rx.iterrows():
        if pd.notnull(row.get("starttime")):
            attrs = {
                "drug": row.get("drug", None),
                "route": row.get("route", None),
                "dose_val_rx": row.get("dose_val_rx", None),
                "dose_unit_rx": row.get("dose_unit_rx", None),
            }
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ANTI_PLT_ORDER",
                "timestamp": row["starttime"],
                "attributes": _attrs_to_json(attrs),
            })
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ANTI_PLT_ADMIN",
                "timestamp": row["starttime"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


# ==============================
# 4. Event Log 통합 함수
# ==============================

def build_event_log(cohort: pd.DataFrame, tables: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    주어진 cohort subset(subject_id, hadm_id)에 대해 Event Log 생성.

    최종 컬럼:
      - case_id
      - subject_id
      - hadm_id
      - event_name
      - timestamp
      - attributes(JSON 문자열)

    로직:
      1) ED를 제외한 모든 이벤트(ADMISSION, ICU, TROP, ECG, PCI, ANTI-PLT)를 먼저 생성
      2) 이 이벤트들의 최소 timestamp와 admissions.admittime을 이용해
         hadm별 '최초 timestamp'를 계산
      3) 실제 ED(ED_ARRIVAL/ED_DEPARTURE)를 edstays에서 생성
      4) ED가 전혀 없는 hadm에 대해서는
         최초 timestamp 기반 ED_ARRIVAL_SURR 생성
      5) DISCHARGE/DEATH 이후 이벤트는 잘라냄
    """
    admissions = tables["admissions"]
    patients = tables["patients"]
    icustays = tables["icustays"]
    edstays = tables["edstays"]
    labevents_trop = tables["labevents_trop"]
    procedures_icd = tables["procedures_icd"]
    prescriptions = tables["prescriptions"]
    ecg = tables["ecg"]

    # 1) ED를 제외한 이벤트 먼저 생성
    adm_events = build_admission_events(cohort, admissions, patients)
    icu_events = build_icu_events(cohort, icustays)
    trop_events = build_troponin_events(cohort, labevents_trop, TROP_POS_THRESHOLD)
    ecg_events = build_ecg_events(cohort, ecg)
    pci_events = build_pci_events(cohort, procedures_icd,
                                  pci_icd9_prefixes=PCI_ICD9_PREFIXES,
                                  pci_icd10_prefixes=PCI_ICD10_PREFIXES)
    antiplatelet_events = build_antiplatelet_events(cohort, prescriptions, ANTI_PLT_DRUGS)

    pre_events = pd.concat(
        [
            adm_events,
            icu_events,
            trop_events,
            ecg_events,
            pci_events,
            antiplatelet_events,
        ],
        ignore_index=True
    )

    # 2) hadm별 최초 timestamp (pre_events 기준)
    if not pre_events.empty:
        pre_events = pre_events.dropna(subset=["timestamp"])
        pre_events["timestamp"] = pd.to_datetime(pre_events["timestamp"], errors="coerce")
        pre_events = pre_events.dropna(subset=["timestamp"])
        earliest_from_events = pre_events.groupby("hadm_id")["timestamp"].min()
    else:
        earliest_from_events = pd.Series(dtype="datetime64[ns]")

    # admissions.admittime 기반 fallback
    adm_subset = admissions.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    adm_subset = _to_datetime(adm_subset, ["admittime"])
    earliest_admit = (
        adm_subset.dropna(subset=["admittime"])
        .groupby("hadm_id")["admittime"].min()
        if not adm_subset.empty else pd.Series(dtype="datetime64[ns]")
    )

    # 3) 실제 ED 이벤트 생성
    real_ed_events = build_real_ed_events(cohort, edstays)
    hadm_with_real_ed = set(real_ed_events["hadm_id"].unique()) if not real_ed_events.empty else set()

    # 4) surrogate ED_ARRIVAL 생성
    surrogate_rows = []
    for _, row in cohort.iterrows():
        sid = row["subject_id"]
        hid = row["hadm_id"]

        # 이미 실제 ED가 있으면 surrogate 생성 안 함
        if hid in hadm_with_real_ed:
            continue

        ts = None
        if hid in earliest_from_events.index:
            ts = earliest_from_events.loc[hid]
        elif hid in earliest_admit.index:
            ts = earliest_admit.loc[hid]

        if ts is None or pd.isna(ts):
            # 완전히 timestamp를 알 수 없는 hadm → 이벤트를 만들 수 없음
            continue

        surrogate_rows.append({
            "subject_id": sid,
            "hadm_id": hid,
            "event_name": "ED_ARRIVAL_SURR",
            "timestamp": ts,
            "attributes": _attrs_to_json({"source": "earliest_timestamp"}),
        })

    surrogate_ed_events = (
        pd.DataFrame(surrogate_rows)
        if surrogate_rows
        else pd.DataFrame(columns=["subject_id", "hadm_id", "event_name", "timestamp", "attributes"])
    )

    # 5) 전체 이벤트 합치기
    all_events = pd.concat(
        [
            pre_events,
            real_ed_events,
            surrogate_ed_events,
        ],
        ignore_index=True
    )

    if all_events.empty:
        return pd.DataFrame(columns=["case_id", "subject_id", "hadm_id", "event_name", "timestamp", "attributes"])

    all_events = all_events.dropna(subset=["timestamp"])
    all_events["timestamp"] = pd.to_datetime(all_events["timestamp"], errors="coerce")
    all_events = all_events.dropna(subset=["timestamp"])

    all_events = all_events.sort_values(
        by=["subject_id", "hadm_id", "timestamp", "event_name"]
    ).reset_index(drop=True)

    # 6) 한 입원(hadm_id)에서 DISCHARGE 또는 DEATH 이후 이벤트 제거
    def trim_after_end(df_one_adm: pd.DataFrame) -> pd.DataFrame:
        end_idx = df_one_adm[df_one_adm["event_name"].isin(["DISCHARGE", "DEATH"])].index
        if len(end_idx) == 0:
            return df_one_adm
        cutoff = end_idx.min()
        return df_one_adm.loc[:cutoff]

    all_events = (
        all_events.groupby("hadm_id", group_keys=False)
                  .apply(trim_after_end)
                  .reset_index(drop=True)
    )

    # case_id = hadm_id
    all_events["case_id"] = all_events["hadm_id"]

    # 최종 컬럼 순서 정리
    all_events = all_events[
        ["case_id", "subject_id", "hadm_id", "event_name", "timestamp", "attributes"]
    ]

    return all_events


def debug_check_ed_icu(cohort: pd.DataFrame, tables: Dict[str, pd.DataFrame]):
    edstays = tables["edstays"]
    icustays = tables["icustays"]

    print("\n[DEBUG] ===== ED / ICU 매칭 상태 점검 =====")

    # 1) ED
    if edstays is None:
        print("[DEBUG][ED] edstays.csv 를 찾지 못했습니다. 실제 ED 이벤트는 생성되지 않습니다.")
        print("            다만 earliest timestamp/admittime 기반 ED_ARRIVAL_SURR 는 생성됩니다.")
    else:
        print(f"[DEBUG][ED] edstays 원본 row 수: {len(edstays)}")
        print(f"[DEBUG][ED] cohort row 수: {len(cohort)}")

        ed_merge = edstays.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
        print(f"[DEBUG][ED] (subject_id, hadm_id) 기준 merge 후 row 수: {len(ed_merge)}")

        if len(ed_merge) > 0:
            print("[DEBUG][ED] 예시 5개:")
            print(ed_merge[["subject_id", "hadm_id", "intime", "outtime"]].head())
        else:
            print("[DEBUG][ED] 매칭된 ED row가 없습니다. 모든 hadm은 surrogate ED_ARRIVAL만 갖게 됩니다.")

    # 2) ICU
    icu = icustays
    print(f"\n[DEBUG][ICU] icustays 원본 row 수: {len(icu)}")
    icu_merge = icu.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    print(f"[DEBUG][ICU] (subject_id, hadm_id) 기준 merge 후 row 수: {len(icu_merge)}")

    if len(icu_merge) > 0:
        print("[DEBUG][ICU] 예시 5개:")
        print(icu_merge[["subject_id", "hadm_id", "intime", "outtime"]].head())
    else:
        print("[DEBUG][ICU] 매칭된 ICU row가 없습니다. 이 경우 ICU_INTIME/OUTTIME 이벤트는 생성되지 않습니다.")


# ==============================
# 5. batch + versioning MAIN
# ==============================

def main():
    # 1) 코호트 로딩 (모든 hadm 사용)
    cohort = load_cohort_all_hadm(COHORT_PATH)
    tables = load_source_tables()

    debug_check_ed_icu(cohort, tables)

    print(f"[INFO] 최종 STEMI cohort size (모든 hadm): {len(cohort)}")
    print(f"[INFO] 최종 고유 hadm 수: {cohort['hadm_id'].nunique()}")

    # 기존 전체 Event Log 파일이 있으면 삭제
    if os.path.exists(EVENT_LOG_FULL_PATH):
        os.remove(EVENT_LOG_FULL_PATH)
        print(f"[INFO] 기존 전체 Event Log 파일 삭제: {EVENT_LOG_FULL_PATH}")

    header_written_full = False
    version = CHUNK_VERSION_START

    n = len(cohort)
    for start in range(0, n, BATCH_SIZE):
        end = min(start + BATCH_SIZE, n)
        sub_cohort = cohort.iloc[start:end].copy()

        print(f"\n[INFO] Batch 처리중: rows {start} ~ {end-1} "
              f"(size={len(sub_cohort)}), version={version}")

        event_log_chunk = build_event_log(sub_cohort, tables)
        num_hadm_chunk = event_log_chunk["hadm_id"].nunique() if not event_log_chunk.empty else 0
        print(f"[INFO]   → 생성된 이벤트 row 수: {len(event_log_chunk)}")
        print(f"[INFO]   → 포함된 hadm_id 수: {num_hadm_chunk}")

        # (1) batch 전용 파일 저장: cohort_ver{version}.csv
        chunk_path = os.path.join(OUTPUT_DIR, f"cohort_ver{version}.csv")
        event_log_chunk.to_csv(chunk_path, index=False)
        print(f"[INFO]   → chunk 파일 저장: {chunk_path}")

        # (2) 전체 Event Log 파일에 append
        if len(event_log_chunk) > 0:
            mode = "a"
            header = not header_written_full
            event_log_chunk.to_csv(
                EVENT_LOG_FULL_PATH,
                mode=mode,
                header=header,
                index=False
            )
            header_written_full = True
            print(f"[INFO]   → 전체 Event Log에 append: {EVENT_LOG_FULL_PATH}")
        else:
            print("[WARN]   → 이 batch에서는 생성된 이벤트가 없습니다.")

        version = 140

    print("\n[INFO] 모든 batch 처리가 완료되었습니다.")
    print(f"[INFO] 최종 전체 Event Log 경로: {EVENT_LOG_FULL_PATH}")
    print(f"[INFO] 마지막 사용된 version 번호: {version}")


if __name__ == "__main__":
    main()


[COHORT] 입력 cohort row 수: 1930
[COHORT] 입력 cohort 고유 subject 수: 1878
[COHORT] 입력 cohort 고유 hadm 수: 1929
[COHORT] (subject_id, hadm_id) 중복 제거 후 row 수: 1929
[COHORT] 최종 고유 subject 수: 1878
[COHORT] 최종 고유 hadm 수: 1929
[LOAD] admissions, patients, icustays...
[LOAD] edstays (있으면 로딩)...
[LOAD] labevents_troponin ...
[LOAD] labevents_troponin.csv 로딩: ../../data/MIMIC4-hosp-icu\labevents_troponin.csv
[LOAD] procedures_icd, prescriptions, ECG machine_measurements...


C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:138: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv(os.path.join(HOSP_DIR, "prescriptions.csv"))
C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:139: DtypeWarning: Columns (16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ecg = pd.read_csv(os.path.join(ECG_DIR, "machine_measurements.csv"))



[DEBUG] ===== ED / ICU 매칭 상태 점검 =====
[DEBUG][ED] edstays 원본 row 수: 425087
[DEBUG][ED] cohort row 수: 1929
[DEBUG][ED] (subject_id, hadm_id) 기준 merge 후 row 수: 1933
[DEBUG][ED] 예시 5개:
   subject_id     hadm_id               intime              outtime
0    10000764  27897940.0  2132-10-14 19:31:00  2132-10-14 23:32:59
1    10010058  26359957.0  2147-11-18 00:50:00  2147-11-18 03:19:00
2    10012438  22764825.0  2178-06-07 19:33:00  2178-06-07 21:57:00
3    10013310  21243435.0  2153-05-26 08:56:00  2153-05-26 14:18:39
4    10013310  27682188.0  2153-05-06 10:21:00  2153-05-06 18:28:00

[DEBUG][ICU] icustays 원본 row 수: 94458
[DEBUG][ICU] (subject_id, hadm_id) 기준 merge 후 row 수: 983
[DEBUG][ICU] 예시 5개:
   subject_id   hadm_id               intime              outtime
0    10010058  26359957  2147-11-18 03:19:00  2147-11-19 08:53:33
1    10012438  22764825  2178-06-07 21:57:00  2178-06-08 15:51:15
2    10013310  27682188  2153-05-06 18:28:00  2153-05-07 20:47:19
3    10015860  24698912  2192

C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:593: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_events = pd.concat(
C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:622: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_events.groupby("hadm_id", group_keys=False)


[INFO]   → 생성된 이벤트 row 수: 2351
[INFO]   → 포함된 hadm_id 수: 100
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver140.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 100 ~ 199 (size=100), version=140


...

[INFO]   → 생성된 이벤트 row 수: 1810
[INFO]   → 포함된 hadm_id 수: 100
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver140.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 1800 ~ 1899 (size=100), version=140


C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:593: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_events = pd.concat(
C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:622: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_events.groupby("hadm_id", group_keys=False)


[INFO]   → 생성된 이벤트 row 수: 1752
[INFO]   → 포함된 hadm_id 수: 100
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver140.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 1900 ~ 1928 (size=29), version=140


C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:593: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_events = pd.concat(
C:\Users\User\AppData\Local\Temp\ipykernel_26072\2319555427.py:622: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_events.groupby("hadm_id", group_keys=False)


[INFO]   → 생성된 이벤트 row 수: 530
[INFO]   → 포함된 hadm_id 수: 29
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver140.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] 모든 batch 처리가 완료되었습니다.
[INFO] 최종 전체 Event Log 경로: ./cohort\event_log_stemi_all.csv
[INFO] 마지막 사용된 version 번호: 139
